In [1]:
import pandas as pd
df=pd.read_csv(r'../notebook/data/gemstone.csv')

In [2]:
df=df.drop(labels=['id'],axis=1)

In [3]:
df.head()

,carat,cut,color,clarity,depth,table,x,y,z,price
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [4]:
X=df.drop(labels=['price'],axis=1)  # Separe Target from features
X


,carat,cut,color,clarity,depth,table,x,y,z
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77
...,...,...,...,...,...,...,...,...,...
193568,0.31,Ideal,D,VVS2,61.1,56.0,4.35,4.39,2.67
193569,0.70,Premium,G,VVS2,60.3,58.0,5.75,5.77,3.47
193570,0.73,Very Good,F,SI1,63.1,57.0,5.72,5.75,3.62
193571,0.34,Very Good,D,SI1,62.9,55.0,4.45,4.49,2.81


In [5]:
y=df[['price']]
y

,price
0,13619
1,13387
2,2772
3,666
4,14453
...,...
193568,1130
193569,2874
193570,3036
193571,681


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193573 entries, 0 to 193572
Data columns (total 10 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   carat    193573 non-null  float64
 1   cut      193573 non-null  object 
 2   color    193573 non-null  object 
 3   clarity  193573 non-null  object 
 4   depth    193573 non-null  float64
 5   table    193573 non-null  float64
 6   x        193573 non-null  float64
 7   y        193573 non-null  float64
 8   z        193573 non-null  float64
 9   price    193573 non-null  int64  
dtypes: float64(6), int64(1), object(3)
memory usage: 14.8+ MB


In [7]:
catgorical_column=X.select_dtypes(include='object').columns
catgorical_column

Index(['cut', 'color', 'clarity'], dtype='object')

In [8]:
numerical_column=X.select_dtypes(exclude='object').columns
numerical_column

Index(['carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')

In [9]:
cut_categories=df['cut'].unique()
color_categories=df['color'].unique()
clarity_categories=df['clarity'].unique()
print(cut_categories)
print(color_categories)
print(clarity_categories)

['Premium' 'Very Good' 'Ideal' 'Good' 'Fair']
['F' 'J' 'G' 'E' 'D' 'H' 'I']
['VS2' 'SI2' 'VS1' 'SI1' 'IF' 'VVS2' 'VVS1' 'I1']


# Now Creating a single Pipeline where we perform

1 : imputer 

2 : encoding 

3 : scaling  


In [11]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [15]:
num_pipeline=Pipeline(   # Numerical Pipeline deals with numerical values
    steps=[
        ('imputer',SimpleImputer(strategy='median')),  # If any Missing Value it handles automatically
        ('scaler',StandardScaler()),
    ]
)
cat_pipeline=Pipeline(  # categorical Pipeline deals with categorical values
    steps=[
        ('imputer',SimpleImputer(strategy="most_frequent")),   # Categorical Missing Values can be handled by mode 
        ('encoding',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
    ]
)

preprocesssor=ColumnTransformer(
    [
        ('num_pipeline',num_pipeline,numerical_column),
        ('categorical_pipeline',cat_pipeline,catgorical_column)
    ]
)

In [16]:
preprocesssor

ColumnTransformer(transformers=[('num_pipeline',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 Index(['carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')),
                                ('categorical_pipeline',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoding',
                                                  OrdinalEncoder(categories=[array(['Premium', 'Very Good', 'Ideal', 'Good', 'Fair'], dtype=object),
                                                                             array(['F', 'J', 'G', 'E', 'D', 'H', 'I'], dtype=object),
                                                                             array(['VS2', 'SI2', 'VS1', 'SI1', 'IF', 'VVS2', 'VVS1', 'I1'],
      dtype=object)]))]),
                                 Index(['cut', 'color', 'clarity'], dtype='object'))])

In [17]:
# Train Test Split
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=30)

In [18]:
preprocesssor.fit_transform(X_train)

array([[-0.97543926, -0.84960654, -0.12153081, ...,  2.        ,
         6.        ,  5.        ],
       [ 0.2351953 ,  1.83363716, -0.12153081, ...,  3.        ,
         3.        ,  3.        ],
       [ 0.49461699,  0.81585507,  0.39980029, ...,  0.        ,
         2.        ,  2.        ],
       ...,
       [ 0.45138004,  1.55606023, -0.6428619 , ...,  3.        ,
         2.        ,  3.        ],
       [ 0.66756478, -1.77486298,  1.44246248, ...,  2.        ,
         2.        ,  2.        ],
       [ 0.25681377,  0.81585507, -0.12153081, ...,  2.        ,
         2.        ,  3.        ]], shape=(135501, 9))

In [ ]:
# Had to do this because of Data Leakeage Ocuuring
X_train=pd.DataFrame(preprocesssor.fit_transform(X_train),columns=preprocesssor.get_feature_names_out())

In [20]:
X_train

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,categorical_pipeline__cut,categorical_pipeline__color,categorical_pipeline__clarity
0,-0.975439,-0.849607,-0.121531,-1.042757,-1.080970,-1.123150,2.0,6.0,5.0
1,0.235195,1.833637,-0.121531,0.318447,0.279859,0.485354,3.0,3.0,3.0
2,0.494617,0.815855,0.399800,0.570855,0.606458,0.673737,0.0,2.0,2.0
3,-1.018676,0.260701,0.921131,-1.214034,-1.244270,-1.195605,0.0,2.0,0.0
4,-0.953821,-0.664555,-0.642862,-1.069801,-1.044681,-1.094168,2.0,1.0,5.0
...,...,...,...,...,...,...,...,...,...
135496,-1.040295,-0.016876,-0.642862,-1.268122,-1.244270,-1.239078,2.0,3.0,3.0
135497,0.991842,0.168176,-0.642862,1.048629,1.114501,1.079486,2.0,2.0,1.0
135498,0.451380,1.556060,-0.642862,0.516768,0.588314,0.702719,3.0,2.0,3.0
135499,0.667565,-1.774863,1.442462,0.868337,0.951202,0.688228,2.0,2.0,2.0
